In [ ]:
from ipywidgets import *
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
import numpy as np
%matplotlib notebook

# First we are going to read in the raw data. It is a txt file with two columns: 
# the real and imaginary parts of each k space point. There are 65536 rows, so it 
# corresponds to 256x256 sampling of k space. 

# First we read in the two columns
data = np.loadtxt('assets/mri-kdata.txt')

# Then reshape into 2D arrays corresponding to the real and imaginary parts of k space
real = np.reshape(data[:,0],(256,256))
imag = np.reshape(data[:,1],(256,256))

# Shift frequencies
real = np.roll(real,32,0)
imag = np.roll(imag,32,0)

# Log transform (add small number to avoid underflow issues)
logreal = np.log(np.abs(real+1e-12))
logimag = np.log(np.abs(imag+1e-12))

f, ax = plt.subplots(1, 3, figsize=(8,8/3))

# Plot original
ax[0].imshow(logreal, cmap='gray')
ax[1].imshow(logimag, cmap='gray')

# Now we combine them into a complex array
kdata = real + 1j * imag

# Compute and plot FT
image = np.abs(np.fft.fftshift(np.fft.ifft2(kdata)))
ax[2].imshow(image, cmap='gray')

ax[0].set_title('Real part k space')
ax[1].set_title('Imag part k space')
ax[2].set_title('Image')

In [ ]:
# Now let's see what happens if we acquire a smaller portion of k space data. 
# TODO: Adjust the truncation width. 

f, ax = plt.subplots(1, 3, figsize=(8,8/3))

# Plot original
ax[0].imshow(image, cmap='gray')

# Let's just use the central 64x64 portion of k space
trunc_width = 64
idx = slice((kdata.shape[0] - trunc_width)//2,(kdata.shape[0] + trunc_width)//2,1)
kdatatrunc = np.zeros_like(kdata)
kdatatrunc[idx,idx] = kdata[idx,idx]

# Plot truncated spectrum
logreal = np.log(np.abs(kdatatrunc) + 1e-12)
ax[1].imshow(logreal, cmap='gray')

# Compute and plot FT
imagetrunc = np.abs(np.fft.fftshift(np.fft.ifft2(kdatatrunc)))
ax[2].imshow(imagetrunc, cmap='gray')

# Labels
ax[0].set_title('Original')
ax[1].set_title('Truncated Spectrum')
ax[2].set_title('Resulting Image')

In [ ]:
# Now let's see what happens if we acquire a less densely sampled set of k space points. 
# TODO: Modify the code to sample less densely along the x direction instead of the y direction. 
# TODO: Modify the code to sample less densely along both the x and y directions. 

f, ax = plt.subplots(1, 3, figsize=(8,8/3))

# Plot original
ax[0].imshow(image, cmap='gray')

# Let's just downsample n y by a factor of 2 
d = 2
kdatasamp = kdata[::d,::1]

logreal = np.log(np.abs(kdatasamp) + 1e-12)
ax[1].imshow(logreal, cmap='gray')

imagesamp = np.abs(np.fft.fftshift(np.fft.ifft2(kdatasamp)))
ax[2].imshow(imagesamp, cmap='gray')

# Labels
ax[0].set_title('Original')
ax[1].set_title('Undersampled Spectrum')
ax[2].set_title('Resulting Image')

In [ ]:
# Add a large electrostatic spike to one k space point

spikekdata = np.copy(kdata)
spikekdata[160,160] = 1.0e1

f, ax = plt.subplots(1, 3, figsize=(8,8/3))

# Plot original
ax[0].imshow(image, cmap='gray')

logreal = np.log(np.abs(spikekdata) + 1e-12)
ax[1].imshow(logreal, cmap='gray')

imagespike = np.abs(np.fft.fftshift(np.fft.ifft2(spikekdata)))
ax[2].imshow(imagespike, cmap='gray')

# Labels
ax[0].set_title('Original')
ax[1].set_title('Spiked Spectrum')
ax[2].set_title('Resulting Image')

In [ ]:
# Add random phase
# TODO: Modify the "strength" of the phase noise.

phasenoisekdata = np.copy(kdata)

for line in range(0,255,1):
    randn = np.random.normal(size=1)
    shift = np.exp(-1j*0.5*randn)
    phasenoisekdata[line,:] = kdata[line,:]*shift

f, ax = plt.subplots(1, 3, figsize=(8,8/3))

# Plot original
ax[0].imshow(image, cmap='gray')

logimag = np.log(np.imag(phasenoisekdata + 1e-12))
ax[1].imshow(logimag, cmap='gray')

imagerand = np.abs(np.fft.fftshift(np.fft.ifft2(phasenoisekdata)))
ax[2].imshow(imagerand, cmap='gray')

# Labels
ax[0].set_title('Original')
ax[1].set_title('Corrupted Spectrum')
ax[2].set_title('Resulting Image')